In [7]:
import sys
import numpy as np
import mdtraj as md

from simtk import unit
from simtk.openmm import MonteCarloAnisotropicBarostat
from simtk.openmm.app import PDBReporter, DCDReporter
from simtk.openmm.app import ForceField, Modeller
from simtk.openmm.app import HBonds, PME
from simtk.openmm.app import PDBFile

from openmmtools import forces
from parmed.openmm.reporters import RestartReporter
from openforcefield.topology import Molecule
from openmmforcefields.generators import GAFFTemplateGenerator
from openforcefield.utils.toolkits import AmberToolsToolkitWrapper

%run utilities
# %run OPLSLennardJones

In [8]:

STEP_SIZE = 250
TOTAL_STEPS = 100
SIM_DURATION = TOTAL_STEPS*STEP_SIZE

LIGAND_OFFSET = (10.0, 10.0, 0.0)

ENERGY_TOLERANCE = 2.0
MAX_ENERGY_ITERS = 5000

CONSTRAINTS = HBonds
NONBONDED_METHOD = PME
NONBONDED_CUTOFF = 1.0*unit.nanometers
SWITCH_DISTANCE = 10.0*unit.angstroms

BARO_FREQUENCY = 25
BARO_PRESSURE = 1.0*unit.bar
BARO_TEMPERATURE = 300.0*unit.kelvin

LANGE_TOLERANCE = 10**-5
LANGE_TEMPERATURE = 300.0*unit.kelvin
LANGE_FRICTION = 1.0/unit.picoseconds
LANGE_STEPSIZE = 2.0*unit.femtoseconds

In [9]:
RENDER_INTERVAL = 5

PDB_FRAME_DIR= 'pdb-frames'
PDB_REPORT_FILE = 'simulation.pdb'
DCD_REPORT_FILE = 'simulation.dcd'
STATE_REPORT_FILE = 'simulation.state.txt'

STEP_PER_REPORT = 100
RESTART_FILE = 'restart.rst7'
STATE_RESTART_FILE = 'restart.xml'
CHECKPOINT_RESTART_FILE = 'restart.chk'

In [10]:
cdtMOL = md.load('mol/di-o-methyl-beta-cyclodextrin-CID10171019.mol2')
cdtMOL_top = cdtMOL.top.to_openmm()
cdtMOL_xyz = cdtMOL.openmm_positions(0)

ligMOL = md.load('mol/ligand-CID644019.mol2')
ligMOL_top = ligMOL.top.to_openmm()
ligMOL_xyz = ligMOL.openmm_positions(0)

ligMOL.xyz += np.array(LIGAND_OFFSET)
# min_ap_dist = ((ligMOL.xyz[0] ** 2.).sum(1) ** 0.5).max() + ((ligMOL.xyz[0] ** 2.).sum(1) ** 0.5).max() + 0.3
# ligMOL.xyz += np.array([1.0, 0.0, 0.0]) * min_ap_dist

model = Modeller(cdtMOL_top, cdtMOL_xyz)
model.add(ligMOL_top, ligMOL_xyz)
# model.addExtraParticles(forcefield)
# model.addHydrogens(forcefield)
# model.addSolvent(forcefield, padding=5*unit.angstroms)

In [12]:
molecules = [
    Molecule.from_file('sdf/ligand-CID644019.sdf'),
    Molecule.from_file('sdf/di-o-methyl-beta-cyclodextrin-CID10171019.sdf')]

for mol in molecules: 
    print(mol)
    mol.generate_conformers(n_conformers=1)
    mol.compute_partial_charges_am1bcc(AmberToolsToolkitWrapper())
    
GAFFForceField = GAFFTemplateGenerator.INSTALLED_FORCEFIELDS[-1]
GAFFTemplate = GAFFTemplateGenerator(forcefield=GAFFForceField)
GAFFTemplate.add_molecules(molecules)

forcefield = ForceField()
forcefield.registerTemplateGenerator(GAFFTemplate.generator)

Molecule with name '644019' and SMILES '[H][O][C]1=[C]([H])[C]([C]([H])([H])[C]([H])([H])[C]([H])([H])[C]([H])([H])[C]([H])([H])[H])=[C]([H])[C]([O][H])=[C]1[C@]1([H])[C]([H])=[C]([C]([H])([H])[H])[C]([H])([H])[C]([H])([H])[C@@]1([H])[C](=[C]([H])[H])[C]([H])([H])[H]'


In [ ]:
print('Creating system...')
system = forcefield.createSystem(
    model.topology, 
    constraints=CONSTRAINTS,
    nonbondedMethod=NONBONDED_METHOD, 
    nonbondedCutoff=NONBONDED_CUTOFF,
    switchDistance=SWITCH_DISTANCE)

print('Adding barostat force...')
system.addForce(
    MonteCarloAnisotropicBarostat(
        defaultPressure=BARO_PRESSURE, defaultTemperature=BARO_TEMPERATURE,
        frequency=BARO_FREQUENCY, scaleX=True, scaleY=True, scaleZ=True))

In [ ]:
integrator = LangevinIntegrator(LANGE_TEMPERATURE, LANGE_FRICTION, LANGE_STEPSIZE)
integrator.setConstraintTolerance(LAGNE_TOLERANCE)

In [ ]:
simulation = Simulation(model.topology, system, integrator) 
simulation.context.setPositions(model.positions)
simulation.minimizeEnergy(ENERGY_TOLERANCE, MAX_ENERGY_ITERS)

simulation.reporters.append(
    PDBReporter(PDB_REPORT_FILE, STEP_PER_REPORT),
    DCDReporter(DCD_REPORT_FILE, STEP_PER_REPORT),
    StateDataReporter(
        STATE_REPORT_FILE, STEP_PER_REPORT, 
        progress=True, temperature=True, 
        potentialEnergy=True, density=True,
        totalSteps=TOTAL_STEPS, speed=True),
    StateDataReporter(
        sys.stdout, STEP_PER_REPORT, 
        progress=True, temperature=True, 
        potentialEnergy=True, density=True,
        totalSteps=TOTAL_STEPS, speed=True)))

In [ ]:
for (i,_) in enumerate(range(STEP_SIZE, SIM_DURATION, STEP_SIZE)): 
    simulation.step(STEP_SIZE)
    if not i + 1 % RENDER_INTERVAL:
        report = PDBFile(PDB_REPORT_FILE)
        topology = report.getTopology()
        positions = report.getPositions(i)
        render_traj(topology, positions)

In [ ]:
if STATE_FILE: simulation.saveState(STATE_FILE)
if CHECKPOINT_FILE: simulation.saveCheckpoint(CHECKPOINT_FILE)
if PDB_FRAME_DIR: save_frames_pdb(PDB_REPORT_FILE, where=PDB_FRAME_DIR, name=PDB_REPORT_FILE)
